## Calculation of R parameter


default: 
- atmosphere = April
- nucleus = p
- hadronic interaction model = Sibyll2.3c


modification parameters:
- nucleus type
- atmosphere
- depth: 1.5 or 3.5
- pi-air cross section factor


In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm
import pickle
import numpy as np

import crflux.models as pm
import mceq_underground_helpers_cs as mh


In [2]:
plot_dir = '/hetghome/khymon/Plots/cs-analysis/'

In [3]:
d = 1.5 #1.5 or 3.5

In [4]:
s_d = mh.slant_depths
angles = mh.angles
x_mod = d/np.cos(np.deg2rad(angles)) # for specific depth

In [5]:
yields_jan_mod = "yields_jan_p" + pi_mod + nucleus
yields_apr_mod = "yields_apr_p" + pi_mod + nucleus
yields_jul_mod = "yields_jul_p" + pi_mod+ nucleus

jan_mod = "jan_p" + pi_mod+ nucleus
jul_mod = "jul_p" + pi_mod+ nucleus

if pi_mod == "15":
    cs = '1.5'
elif pi_mod == "05":
    cs = '0.5'

# make unmodified label including option to add iron

yields_jan = "yields_jan" + nucleus
yields_apr = "yields_apr" + nucleus
yields_jul = "yields_jul" + nucleus

jan = "jan" + nucleus
jul = "jul" + nucleus


NameError: name 'pi_mod' is not defined

In [21]:
m = mh.n_mu_vec # muon multiplicity

In [6]:
def X(d):
    s_d = mh.slant_depths
    angles = mh.angles
 
    return d/np.cos(np.deg2rad(angles))
    

In [12]:
def dNmu_dmu(d,month, ptype, cs): # month = str
    angle = 0.0

    #get depth according to angle
    x_mod = X(d)

    #get number of muons per multiplicity
    dNmudmu = np.ndarray(shape=(len(x_mod),101),dtype='float')     
    for i in range(len(x_mod)):

        dNmudmu[i] = 1e4*mh.mult_dist(x_mod[i],
                                    0.,
                                    pm.GlobalSplineFitBeta(),"yields_" +month,
                                    ptype,
                                    cs,
                                    norm=False
                                ) / mh.rates(x_mod[i], angle, month, ptype, cs)
    return dNmudmu
    

In [15]:
# function to calculate R from dN_mu from unmodified and modofied cs 
def R(m,dN_dNmu):
    int_low = np.zeros(len(dN_dNmu[:,0]))
    int_high = np.zeros(len(dN_dNmu[:,0]))
    for i in range(len(dN_dNmu[:,0])):
        int_low = np.trapezoid(dN_dNmu[i,1:19],m[1:19])
        int_high = np.trapezoid(dN_dNmu[i,59:],m[59:])    

    return int_high / int_low



In [17]:
def R_normalized(R_mod,d,ptype):
    # default parameters
    dNu_dmu_apr = dNmu_dmu(d,month="apr", ptype=ptype, cs=1.0) #default cs
    R_def_apr = R(m,dNu_dmu_apr)
    
    return R_mod/R_def_apr

In [18]:
dNu_dmu_jul = dNmu_dmu(d,month="jul", ptype=2212, cs=1.0)

In [19]:
dNu_dmu_jul_mod = dNmu_dmu(d,month="jul", ptype=2212, cs=1.5)


In [13]:
x_mod

array([1.875     , 1.66666667, 1.5       ])

In [22]:

R_def_jul = R(m,dNu_dmu_jul)

In [15]:
R(m,dNu_dmu_jul)/R(m,dNu_dmu_jan)

np.float64(1.122416688953364)

In [24]:

R_normalized(R_def_jul,x_mod,2212)

np.float64(1.0494233817275467)

In [17]:
R_05_p_jan = R_normalized(R(m,dNu_dmu_jan))

TypeError: R_normalized() missing 1 required positional argument: 'x_mod'